##### This notebook imports source data for this project into BigQuery. 
##### The source data is first exported in GCS in CSV format. The BigQuery source table is "bigquery-public-data.ml_datasets.ulb_fraud_detection". This is a table of credit card transactions that are classified as fradulant, Class =1 or normal, Class = 0.

##### This table has 284807 credit card transaction classified as fradulent or normal in the column Class. In order to protect confidentiality the original features have been transformed using PCA(Principal component analysis) into 28 features named V1,V2,....,V28(float). 
##### Two descriptive features are provided without transformation by PCA.
##### Time (integer): is the seconds elapsed between the transaction and the eraliest transaction in the table
##### Amount (float): is the value of the transaction.

#### Preparation of Data
##### transaction_id (string): a unique id for row/transaction
##### splits (string): this divides the transactions into the set of TRAIN(80%), VALIDATE(10%), TEST(10%)

In [2]:
# Setup
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'ml-pipeline-project-401216'

In [3]:
REGION = "us-central1"
EXPERIMENT = "01"
SERIES = "01"

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = "fraud"
BQ_TABLE = "fraud"

# sourcing data from bigquery-public-data.ml_datasets.ulb_fraud_detection
BQ_SOURCE = "bigquery-public-data.ml_datasets.ulb_fraud_detection"

In [4]:
# packages and clients:
from google.cloud import storage
from google.cloud import bigquery

gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

In [5]:
BUCKET = PROJECT_ID + '-bucket1'
BUCKET

'ml-pipeline-project-401216-bucket1'

In [6]:
# Source data in GCS Storage bucket

file = "{}/{}/data/{}.csv".format(SERIES, EXPERIMENT, BQ_TABLE)
file

'01/01/data/fraud.csv'

In [7]:
# Exporting the dataset into GCS bucket

bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print("The file is already created at: gs://{}/{}".format(bucketDef.name, file))
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = ["gs://{}/{}".format(bucketDef.name, file)])
    print("Creating the export....")
    extract.result()
    print("Exported the table to gs://{}/{}".format(bucketDef.name, file))
    

Creating the export....
Exported the table to gs://ml-pipeline-project-401216-bucket1/01/01/data/fraud.csv


In [8]:
# list files in the bucket
list(bucketDef.list_blobs(prefix = "{}/{}".format(SERIES, EXPERIMENT)))

[<Blob: ml-pipeline-project-401216-bucket1, 01/01/data/fraud.csv, 1697460422115735>]

In [9]:
# Checking if datasets exists in bigQuery
datasets = list(bq.list_datasets())
for sets in datasets:
    print(sets.dataset_id)

In [10]:
# create dataset if missing

ds = bigquery.Dataset("{}.{}".format(BQ_PROJECT ,BQ_DATASET))
ds.location = REGION
ds.labels = {'experiment' : f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

In [11]:
# creating table in the dataset

from google.cloud.exceptions import NotFound

try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    
except NotFound as error:
    print("Creating table....")
    destination = bigquery.TableReference.from_string("{}.{}.{}".format(BQ_PROJECT, BQ_DATASET, BQ_TABLE))
    
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment' : f'{EXPERIMENT}'}
    )
        
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print("Table created successfully: {}.{}.{}".format(BQ_PROJECT, BQ_DATASET, BQ_TABLE))
    

Creating table....
Table created successfully: ml-pipeline-project-401216.fraud.fraud


In [12]:
# Viewing sample of table from bigquery

query = f"""
    SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
    #LIMIT 5
"""

bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.0,0
1,14332,1.071950,0.340678,1.784068,2.846396,-0.751538,0.403028,-0.734920,0.205807,1.092726,...,-0.169632,-0.113604,0.067643,0.468669,0.223541,-0.112355,0.014015,0.021504,0.0,0
2,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,0.125514,0.480049,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0
3,35799,-0.769798,0.622325,0.242491,-0.586652,0.527819,-0.104512,0.209909,0.669861,-0.304509,...,0.152738,0.255654,-0.130237,-0.660934,-0.493374,0.331855,-0.011101,0.049089,0.0,0
4,36419,1.047960,0.145048,1.624573,2.932652,-0.726574,0.690451,-0.627288,0.278709,0.318434,...,0.078499,0.658942,-0.067810,0.476882,0.526830,0.219902,0.070627,0.028488,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142461,72417,-1.403328,0.674555,0.819995,0.834211,0.473596,-0.300257,0.244709,0.382572,-0.586363,...,0.161485,0.647793,-0.215866,0.045493,-0.269521,-0.331446,0.295194,-0.090043,29.8,0
142462,155177,0.177782,1.009887,-0.697991,0.561253,1.129277,-1.318276,1.618830,-0.428442,-0.709535,...,0.302211,0.873776,-0.054461,-0.135198,-0.745705,-0.621537,0.249712,0.266457,29.8,0
142463,68407,-1.221122,0.915613,1.593263,0.034324,-0.577822,0.301421,-0.220222,0.482548,0.340628,...,0.026030,0.014402,-0.186312,-0.412978,-0.168158,0.285696,-0.397185,0.054202,29.8,0
142464,75698,1.146227,0.068700,0.067165,0.947095,0.117907,0.294176,-0.111480,0.191565,-0.052796,...,0.106122,0.242205,-0.185146,-0.683298,0.613173,-0.224871,0.017395,0.003444,29.8,0


In [13]:
# Reviwing data in Bigquery

query = f"""   
    SELECT CLASS FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

df = bq.query(query = query).to_dataframe()
df['CLASS'].value_counts()

0    284315
1       492
Name: CLASS, dtype: Int64

In [14]:
df['CLASS'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: CLASS, dtype: Float64

In [15]:
# Preparing data for Analysis
# create a prepped version of data with test/train splits using SQL DDL

query = f"""
    CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
    WITH add_id AS (SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
    SELECT *,
        CASE
            WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN"
            WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
            ELSE "TEST"
        END AS splits
    FROM add_id
"""

job = bq.query(query = query)
job.result()

In [16]:
(job.ended - job.started).total_seconds()

8.39

In [17]:
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

70.632136 MB


In [18]:
query = f"""
    SELECT splits, count(*) AS Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    GROUP BY splits
"""

bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,VALIDATE,28339,9.950247
1,TEST,28726,10.086129
2,TRAIN,227742,79.963624


In [19]:
query = f"""
    SELECT * FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
    LIMIT 5
"""

data = bq.query(query = query).to_dataframe()

In [20]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,48571,1.318492,-0.847201,1.249334,-0.559317,-1.750040,-0.326865,-1.332506,0.203222,-0.208361,...,-0.069266,0.558256,0.367075,-0.071693,0.041415,0.013417,0.0,0,ef8df9d8-a7c1-4279-9858-8db2eee0c5f0,TEST
1,72182,1.089390,0.359016,1.779216,2.826392,-0.956163,-0.133455,-0.465276,0.062176,-0.104355,...,0.097091,0.938991,0.266291,-0.079971,0.051442,0.043544,0.0,0,e5c69e9a-eff6-47e6-953a-7cfb521a9282,TEST
2,129313,-0.852350,0.875932,0.925625,2.633341,1.201244,2.132422,-0.452718,1.198897,-1.059305,...,0.040960,-0.603421,-1.039566,0.014627,0.157820,0.118013,0.0,0,bce9590b-e156-43c2-92b4-05f0cdb0c78f,TEST
3,75592,-0.699908,1.480374,1.107936,2.643082,0.274661,0.125801,0.330534,0.301284,-1.581983,...,0.007270,0.000673,-0.413005,0.170046,0.007996,-0.084747,0.0,0,19c62da5-c07b-464f-8d3f-1652e3fce5a7,TEST
4,44022,1.171138,-0.039133,0.979372,0.820066,-0.221879,1.181843,-0.900047,0.419782,0.351957,...,-0.196327,-1.122629,0.442433,-0.155258,0.092801,0.018359,0.0,0,3772e22f-db7a-4a73-9a56-def27fa6f521,TEST
